# Convert py2 pytorch checkpoint file to py3
Each step can be run separately.

## STEP 0: Import and read config

In [1]:
from preprocessing import full_prep
from config_submit import config as config_submit

import torch
from torch.nn import DataParallel
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable

from layers import acc
from data_detector import DataBowl3Detector,collate
from data_classifier import DataBowl3Classifier

from utils import *
from split_combine import SplitComb
from test_detect import test_detect
from importlib import import_module
import pandas

datapath = config_submit['datapath']
prep_result_path = config_submit['preprocess_result_path']
skip_prep = config_submit['skip_preprocessing']
skip_detect = config_submit['skip_detect']

c:\users\hasee\appdata\local\programs\python\python35\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [3]:
import pickle
import torch

In [3]:
checkpoint = torch.load(config_submit['classifier_param'])

UnicodeDecodeError: 'ascii' codec can't decode byte 0x8c in position 16: ordinal not in range(128)

In [4]:
# pkl_file = open("./model/c_new.pkl", "rb")
# data = pickle.load(pkl_file, encoding="latin1")

In [5]:
# pkl_file.close()

In [6]:
with open('./model/c_new.pkl', 'rb') as f:
    data_dict = pickle.load(f, encoding='bytes')

In [7]:
data_dict.keys()

dict_keys([b'save_dir', b'args', b'state_dict', b'epoch'])

In [7]:
state_dict_old = data_dict['state_dict'.encode()]

In [27]:
state_dict_old_ = dict(state_dict_old)

In [ ]:
new_dict = {}
for k in state_dict_old_:
    new_key = k.decode()
    new_dict[new_key] = state_dict_old_[k]

In [ ]:
OrderedDict((newkey if k == oldkey else k, v) for k, v in state_dict_old.viewitems())

In [8]:
import torch

In [41]:
torch.save(data_dict, "./model/c_new.ckpt")

In [9]:
cp = torch.load('./model/classifier.ckpt')

In [ ]:
cp['state_dict'] = cp[b'state_dict']
cp.pop(b'state_dict')
pass

In [14]:
cp.keys()

dict_keys([b'save_dir', b'args', b'epoch', 'state_dict'])

In [32]:
# cp[b'save_dir'] = 'results/net4_full'
cp[b'save_dir']

'results/net4_full'

In [33]:
cp['save_dir'] = cp[b'save_dir']
cp.pop(b'save_dir')

'results/net4_full'

In [23]:
cp['save_dir'] = cp[b'save_dir']
cp.pop(b'save_dir')
cp['args'] = cp[b'args']
cp.pop(b'args')
cp['epoch'] = cp[b'epoch']
cp.pop(b'epoch')

160

In [34]:
cp.keys()

dict_keys(['save_dir', 'epoch', 'args', 'state_dict'])

In [36]:
torch.save(cp, "./model/classifier.ckpt")

## Give it a try

In [37]:
casemodel = import_module(config_submit['classifier_model'].split('.py')[0])
casenet = casemodel.CaseNet(topk=5)
config2 = casemodel.config
checkpoint = torch.load(config_submit['classifier_param'])
casenet.load_state_dict(checkpoint['state_dict'])

In [5]:
from config_submit import config as config_submit
import torch
from data_detector import DataBowl3Detector,collate

from importlib import import_module

casemodel = import_module(config_submit['classifier_model'].split('.py')[0])
casenet = casemodel.CaseNet(topk=5)
checkpoint = torch.load(config_submit['classifier_param'])
casenet.load_state_dict(checkpoint['state_dict'])

In [8]:
casenet = casenet.cpu()

## `DataBowl3Classifier` in `data_classifier.py`
Get top 5 for 'test' or sample 5 for training.

In [2]:
datapath = config_submit['datapath']

In [6]:
import os
testsplit = os.listdir(datapath)
testsplit

['0015ceb851d7251b8f399e39779d1e7d', '0030a160d58723ff36d73f41b170ec21']

In [10]:
config2 = casemodel.config
bbox_result_path = './bbox_result'
if not os.path.exists(bbox_result_path):
    os.mkdir(bbox_result_path)

prep_result_path = config_submit['preprocess_result_path']
config2['bboxpath'] = bbox_result_path
config2['datadir'] = prep_result_path
config2

{'T': 1,
 'anchors': [10, 30, 60],
 'augtype': {'flip': True, 'rotate': False, 'scale': False, 'swap': False},
 'bboxpath': './bbox_result',
 'conf_th': -1,
 'crop_size': [64, 64, 64],
 'datadir': 'F:\\LargeFiles\\lfz\\prep_result_sub\\',
 'detect_th': 0.05,
 'filling_value': 160,
 'isScale': True,
 'jitter_range': 0.15,
 'labelfile': '../stage1_labels.csv',
 'lr': [0.01, 0.001, 0.0001, 1e-05],
 'lr_stage': array([ 50, 100, 140, 160]),
 'miss_ratio': 1,
 'miss_thresh': 0.03,
 'nms_th': 0.05,
 'padmask': False,
 'preload_train': True,
 'preload_val': True,
 'radiusLim': [6, 100],
 'random_sample': True,
 'resample': None,
 'scaleLim': [0.85, 1.15],
 'startepoch': 20,
 'stride': 4,
 'topk': 5}

In [11]:
from data_classifier import DataBowl3Classifier

dataset = DataBowl3Classifier(testsplit, config2, phase = 'test')

In [14]:
d0 = dataset[0]

In [15]:
len(d0)

2

### Input crop

In [16]:
d0[0].shape

torch.Size([5, 1, 64, 64, 64])

### coord
* 5, - top 5
* 3, - 
* 16, 
* 16,
* 16

In [17]:
d0[1].shape

torch.Size([5, 3, 16, 16, 16])

In [30]:
d0[1][0,:,:,:,:].max()

tensor(0.3312)

In [ ]:
coord = Variable(coord) # TODO .cuda()
x = Variable(x) # TODO .cuda()

In [32]:
#         xlist: n x k x 1x 96 x 96 x 96
#         coordlist: n x k x 3 x 24 x 24 x 24
xsize = xlist.size()
corrdsize = coordlist.size()
# Reshape the tensor to remove the 3rd axis.
xlist = xlist.view(-1,xsize[2],xsize[3],xsize[4],xsize[5])
coordlist = coordlist.view(-1,corrdsize[2],corrdsize[3],corrdsize[4],corrdsize[5])
casenet.NoduleNet()

Net(
  (preBlock): Sequential(
    (0): Conv3d(1, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv3d(24, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (4): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
  )
  (forw1): Sequential(
    (0): PostRes(
      (conv1): Conv3d(24, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (bn2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv3d(24, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1))
        (1): BatchNorm3d(32, eps=1e-05, 

In [33]:
prep_result_path

'F:\\LargeFiles\\lfz\\prep_result_sub\\'